In [1]:
import sys
sys.float_info
from numba import jit, cuda 
#Should reveal that floats have 8 bytes

In [2]:
MD = "metaData"
Indep = "trainIndependents"
Dep = "trainDependent"
fileNames = [MD, Indep, Dep]

In [7]:
import math ##For basic mathematical operations
import matplotlib.pyplot as plt
import dask.dataframe as dd  
from IPython import display ## Plot setup for Ipython
from matplotlib import cm ##  Colormap reference
from matplotlib import gridspec ##plot setups
from matplotlib import pyplot as plt ##plot setups
import numpy as np 
import pandas as pd
from sklearn.linear_model import LinearRegression
from tqdm import tqdm
import collections
import warnings
from dask.distributed import Client
# warnings.filterwarnings("ignore") - may want to ignore, if know what you are doing, but can be helpful
# warnings.filterwarnings(action = "once") - may want to show only once
# from Queue import priority_queue

In [8]:
client = Client()
client
pd.options.mode.chained_assignment = None

Client Scheduler: tcp://127.0.0.1:40871 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 12 Memory: 16.65 GB


In [9]:
#takes about 1:30 for me
frames = []

for index, fileName in tqdm(enumerate(fileNames)):
    frames.append(dd.read_csv("./TestData/" + fileName + ".csv", sep=",", sample = 2000000))

0it [00:00, ?it/s]distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://127.0.0.1:52556 remote=tcp://127.0.0.1:40871>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://127.0.0.1:52558 remote=tcp://127.0.0.1:40871>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://127.0.0.1:52560 remote=tcp://127.0.0.1:40871>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://127.0.0.1:52562 remote=tcp://127.0.0.1:40871>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://127.0.0.1:52564 remote=tcp://127.0.0.1:40871>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://127.0.0.1:52566 remote=tcp://127.0.0.1:40871>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://127.0.0.1:52568 remote=tcp://127.0.0.1:40871>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://127.0.0.1:52570 remote=tcp://127.0.0

distributed.utils_perf - WARNING - full garbage collections took 21% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 21% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 21% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 21% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 21% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 20% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 20% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 19% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 19% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took

In [10]:
#about 8 seconds
for i in tqdm(range(len(frames))):
    frames[i] = frames[i].fillna(frames[i].mean())

100%|██████████| 3/3 [00:20<00:00,  6.75s/it]


In [11]:
#15 seconds
frames[0].compute()

,NSymbols,NTimes,NIndependent
0,1154,42089,20


In [15]:
frames[1].compute()

distributed.utils_perf - WARNING - full garbage collections took 24% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 23% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 23% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 23% CPU time recently (threshold: 10%)
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://127.0.0.1:52850 remote=tcp://127.0.0.1:40871>


KeyboardInterrupt: 

In [12]:
frames[2].compute()

,Time,Symbol 0000 Dependent,Symbol 0001 Dependent,Symbol 0002 Dependent,Symbol 0003 Dependent,Symbol 0004 Dependent,Symbol 0005 Dependent,Symbol 0006 Dependent,Symbol 0007 Dependent,Symbol 0008 Dependent,...,Symbol 1144 Dependent,Symbol 1145 Dependent,Symbol 1146 Dependent,Symbol 1147 Dependent,Symbol 1148 Dependent,Symbol 1149 Dependent,Symbol 1150 Dependent,Symbol 1151 Dependent,Symbol 1152 Dependent,Symbol 1153 Dependent
0,20070103_093000,-0.000086,-0.000351,-0.000231,-0.00041,0.000528,0.000501,0.000124,0.000053,0.000031,...,-0.016802,0.001788,NaN,0.000365,-0.000752,-0.004589,NaN,NaN,NaN,0.000246
1,20070103_100000,-0.000086,-0.000351,-0.000231,-0.00041,0.000528,0.000501,0.000124,0.000053,0.000031,...,-0.016802,0.001788,NaN,0.000365,-0.000752,-0.004589,NaN,NaN,NaN,0.000246
2,20070103_103000,-0.000086,-0.000351,-0.000231,-0.00041,0.000528,0.000501,0.000124,0.000053,0.000031,...,-0.016802,0.001788,NaN,0.000365,-0.000752,-0.004589,NaN,NaN,NaN,0.000246
3,20070103_110000,-0.000086,-0.000351,-0.000231,-0.00041,0.000528,0.000501,0.000124,0.000053,0.000031,...,-0.016802,0.001788,NaN,0.000365,-0.000752,-0.004589,NaN,NaN,NaN,0.000246
4,20070103_113000,-0.000086,-0.000351,-0.000231,-0.00041,0.000528,0.000501,0.000124,0.000053,0.000031,...,-0.016802,0.001788,NaN,0.000365,-0.000752,-0.004589,NaN,NaN,NaN,0.000246
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1457,20191129_103000,-0.000086,-0.000351,-0.000231,-0.00041,0.000528,0.000501,0.000124,0.000053,0.000031,...,-0.016802,0.001788,NaN,0.000365,-0.000752,-0.004589,NaN,NaN,NaN,0.000246
1458,20191129_110000,-0.000086,-0.000351,-0.000231,-0.00041,0.000528,0.000501,0.000124,0.000053,0.000031,...,-0.016802,0.001788,NaN,0.000365,-0.000752,-0.004589,NaN,NaN,NaN,0.000246
1459,20191129_113000,-0.000086,-0.000351,-0.000231,-0.00041,0.000528,0.000501,0.000124,0.000053,0.000031,...,-0.016802,0.001788,NaN,0.000365,-0.000752,-0.004589,NaN,NaN,NaN,0.000246
1460,20191129_120000,-0.000086,-0.000351,-0.000231,-0.00041,0.000528,0.000501,0.000124,0.000053,0.000031,...,-0.016802,0.001788,NaN,0.000365,-0.000752,-0.004589,NaN,NaN,NaN,0.000246
